### Assignment. 산학지형 기상 정보 대비 Burned Area에 대한 EDA 및 Regression를 활용한 예측
- 데이터 집합 소스
  - 설명: https://archive.ics.uci.edu/ml/datasets/Forest+Fires
    - Features
      - 1: X - x-axis spatial coordinate within the Montesinho park map: 1 to 9 
      - 2: Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9 
      - 3: month - month of the year: 'jan' to 'dec' 
      - 4: day - day of the week: 'mon' to 'sun' 
      - 5: FFMC - FFMC index from the FWI system: 18.7 to 96.20 
      - 6: DMC - DMC index from the FWI system: 1.1 to 291.3 
      - 7: DC - DC index from the FWI system: 7.9 to 860.6 
      - 8: ISI - ISI index from the FWI system: 0.0 to 56.10 
      - 9: temp - temperature in Celsius degrees: 2.2 to 33.30 
      - 10: RH - relative humidity in %: 15.0 to 100 
      - 11: wind - wind speed in km/h: 0.40 to 9.40 
      - 12: rain - outside rain in mm/m2 : 0.0 to 6.4 
    - Target 
      - area - the burned area of the forest (in ha): 0.00 to 1090.84
  - 관련 논문 및 자료
    - P. Cortez and A. Morais. A Data Mining Approach to Predict Forest Fires using Meteorological Data. In J. Neves, M. F. Santos and J. Machado Eds., New Trends in Artificial Intelligence, Proceedings of the 13th EPIA 2007 - Portuguese Conference on Artificial Intelligence, December, Guimarães, Portugal, pp. 512-523, 2007. APPIA, ISBN-13 978-989-95618-0-9. http://www3.dsi.uminho.pt/pcortez/fires.pdf
    - 논문 발췌 중요 그림
    ![attributes](./figures/fire.png)
    ![map](./figures/fire_map.png)
    - Fire Weather Index
      - https://www.frames.gov/files/6014/1576/1411/FWI-history.pdf
  - 데이터: https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv

In [1]:
import urllib2
from scipy import stats
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

path = 'http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
raw_csv = urllib2.urlopen(path)
feature_names = ('buying', 'maint', 'doors', 'persons', "log_boot", "safety")
target_name = 'eval'
all_names = feature_names + (target_name,)
df = pd.read_csv(raw_csv, names=all_names)

In [2]:
dfb

,buying,maint,doors,persons,log_boot,safety,eval
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


### [Mission 1] 모든 속성들을 수치형 데이터로 변환
- Mushroom 데이터 집합에 대한 "Categorical Attribute를 Numerical Attribute로 변환" 강의 내용 참조
- 최종적으로 DataFrame.describe() 함수를 통하여 각 속성별로 기본 통계치를 산출

### [Mission 2] Target 4개별로 그룹핑하여 각 속성별 기본 통계치 및 Box Plot 산출  
- Mushromm 데이터 집합에 대한 "Edible Mushrooms과 Poisonous Mushrooms 의 두 개의 그룹핑 작업 및 각 그룹별 Boxplot 그리기" 강의 내용 참조

### [Mission 3] 각 속성들을 두 개씩 쌍을 지어서 각 쌍마다 Scatter Plot 그리기
- Iris 데이터 집합에 대한 "탐색적 자료 분석 (Exploratory data analysis)" 강의 내용 참조
- 총 6개의 Feature가 존재하므로 두 개씩 쌍을 만든다면 총 15개의 쌍이 나옴
  - 예. (buying, maint), (buying, doors), ...
- Categorical Data를 Numerical Attribute로 변환한 DataFrame으로 작업해야 함
- 그러므로, 총 15개의 Scatter Plot을 산출해야 함
- 15개의 Scatter Plot을 눈으로 분석하여 Guess 할 수 있는 분석 내용을 작성

### [Mission 4] 전체 데이터 집합에 대한 Entropy 산출
- Binary Target 이 아닌 4개의 Target 이 존재함을 유의해야 함

### [Mission 5] Decision Tree 모델 구축을 고려하여 루트노드에서 첫번째 Branch를 위한 IG가 가장 높은 속성 산출 
- 해당 속성을 잘 기억하고 Mission 6에서의 결과와 동일한지 확인해야 함

### [Mission 6] sklearn 모듈을 활용하여 Decision Tree 모델을 구축하고 예측 시도
- 루트노드에서 첫번째 Branch를 구성할 때 활용한 속성이 Mission 5에서 산출한 것과 동일한지 파악
  - 해당 속성에 대하여 sklearn 모듈 내에서 어떠한 방법으로 기준을 정했는지 판단 및 해석
- 모델 구축 후 기존 Training Data중 일부에 대한 예측
- 기존 Training Data에 존재하지 않는 새로운 데이터에 대한 예측